In [36]:
import os
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import os
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
import keras.layers as L
from keras import optimizers, losses, metrics, Model
from keras.callbacks import EarlyStopping
from keras.models import Sequential, load_model
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

In [7]:
inputPath = "D:\Programing\Projects\ESP32-TFLite-Knock-Detector\Data Collection\Data"

commands = np.array(tf.io.gfile.listdir(str(inputPath)))
print(commands)
N_CLASSES = len(commands)
print(N_CLASSES)

['False' 'True']
2


In [41]:
filesArray = []
for path, subdirs, files in os.walk(inputPath):
    for name in files:
        filesArray.append(os.path.join(path + "\\", name))
print(len(filesArray))

for i in filesArray:
    print(i)

563
D:\Programing\Projects\ESP32-TFLite-Knock-Detector\Data Collection\Data\False\data_20230510_191958.txt
D:\Programing\Projects\ESP32-TFLite-Knock-Detector\Data Collection\Data\False\data_20230510_192022.txt
D:\Programing\Projects\ESP32-TFLite-Knock-Detector\Data Collection\Data\False\data_20230510_192026.txt
D:\Programing\Projects\ESP32-TFLite-Knock-Detector\Data Collection\Data\False\data_20230510_192043.txt
D:\Programing\Projects\ESP32-TFLite-Knock-Detector\Data Collection\Data\False\data_20230510_192126.txt
D:\Programing\Projects\ESP32-TFLite-Knock-Detector\Data Collection\Data\False\data_20230510_192132.txt
D:\Programing\Projects\ESP32-TFLite-Knock-Detector\Data Collection\Data\False\data_20230510_192140.txt
D:\Programing\Projects\ESP32-TFLite-Knock-Detector\Data Collection\Data\False\data_20230510_192153.txt
D:\Programing\Projects\ESP32-TFLite-Knock-Detector\Data Collection\Data\False\data_20230510_192209.txt
D:\Programing\Projects\ESP32-TFLite-Knock-Detector\Data Collection\Da

In [47]:
dfs = []
for filename in filesArray:
    df = pd.read_csv(filename, header=None, names=['value'])
    df['filename'] = filename.split("\\")[-2]
    dfs.append(df)
print(dfs[500]["filename"])

0      True
1      True
2      True
3      True
4      True
       ... 
995    True
996    True
997    True
998    True
999    True
Name: filename, Length: 1000, dtype: object


0      D:\Programing\Projects\ESP32-TFLite-Knock-Dete...
1      D:\Programing\Projects\ESP32-TFLite-Knock-Dete...
2      D:\Programing\Projects\ESP32-TFLite-Knock-Dete...
3      D:\Programing\Projects\ESP32-TFLite-Knock-Dete...
4      D:\Programing\Projects\ESP32-TFLite-Knock-Dete...
                             ...                        
995    D:\Programing\Projects\ESP32-TFLite-Knock-Dete...
996    D:\Programing\Projects\ESP32-TFLite-Knock-Dete...
997    D:\Programing\Projects\ESP32-TFLite-Knock-Dete...
998    D:\Programing\Projects\ESP32-TFLite-Knock-Dete...
999    D:\Programing\Projects\ESP32-TFLite-Knock-Dete...
Name: filename, Length: 1000, dtype: object


In [4]:
def get_waveform_and_label(filename):
    label = tf.strings.split(filename, "/")[-2]
    label = tf.argmax(label == commands)
    label = tf.one_hot(label, N_CLASSES)
    #data = tf.io.read_file(filename)
    data = tf.data.TextLineDataset(filename).map(lambda x: tf.strings.to_number(tf.strings.split([x]).values, out_type=float))
    return data, label

def load_dataset(filenames):
    dataset = tf.data.Dataset.from_tensor_slices(filenames)
    return dataset

In [5]:
#data, label = get_waveform_and_label(filesArray)

In [23]:
def get_dataset(fileNames, batch_size=10):  # 256

    AUTO = tf.data.AUTOTUNE
    label = np.array([])
    start = False

    dataset = load_dataset(fileNames)
    dataset = dataset.shuffle(150000)  # dataset_len
    dataset = dataset.map(get_waveform_and_label, num_parallel_calls=AUTO)

    dataset_len = tf.data.experimental.cardinality(dataset).numpy()
    origional = dataset
    
    dataset = origional.take(int((0.95*dataset_len)))
    val_ds = origional.skip(int(0.95*dataset_len))

    dataset = dataset.batch(batch_size)
    val_ds = val_ds.batch(batch_size)
    dataset.repeat()
    dataset = dataset.prefetch(AUTO)

    return dataset, val_ds

dataset, val_ds = get_dataset(filesArray, 5)

 

In [38]:


model = Sequential()
model.add(L.Input((1000,1), name='input_accel'))

# Rebuild Top
model.add(L.Conv1D(10,10))
#model.add(L.BatchNormalization())
#model.add(L.Dropout(0.2))
model.add(L.Dense(32, activation="sigmoid"))
model.add(L.Dense(N_CLASSES, activation="softmax"))

model.compile(optimizer=tf.optimizers.Adam(),
              loss=losses.CategoricalCrossentropy(),
              metrics=[metrics.CategoricalAccuracy()])
# model.build((235, 4001, 1))
print(model.summary())
one_batch = dataset.next()
model.fit(one_batch,
          epochs=5,
          validation_data=val_ds)

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_5 (Conv1D)           (None, 991, 10)           110       
                                                                 
 dense_8 (Dense)             (None, 991, 32)           352       
                                                                 
 dense_9 (Dense)             (None, 991, 2)            66        
                                                                 
Total params: 528
Trainable params: 528
Non-trainable params: 0
_________________________________________________________________
None


AttributeError: 'PrefetchDataset' object has no attribute 'next'